In [26]:
import numpy as np
import pandas as pd
from numpy.random import uniform, normal  
import math
from math import pi
from scipy.optimize import Bounds
from scipy.optimize import minimize

In [27]:
def J_square(sigma_square,tau_square):
    return tau_square/(tau_square+sigma_square)

In [28]:
# k: the number of studies
k_list = [10,30,50,100]
# each element of `parameter_constellation` has the form (mu, sigma^2,tau^2) 
# where tau^2 is the between-study variance, sigma^2*u_i is the within-study variance
parameter_constellation = [(0,12,4),(0,9,4),(0,4,4),(0,2,6)]
# heterogeniety measure: J^2
J_square_list = list()
for element in parameter_constellation:
    sigma_square_i = element[1]
    tau_square_i = element[2]
    J_square_list.append(J_square(sigma_square_i,tau_square_i))
print('J square =',J_square_list)

J square = [0.25, 0.3076923076923077, 0.5, 0.75]


## method1: trying to minimize the negative log likelihood

### The negative log-likelihood function of $v = (\mu, \sigma^2, \tau^2)$ is given by

$I(v) = \frac{1}{2}\sum \limits_{i=1}^{k}[\frac{(D_i-\mu)^2}{\tau^2+\sigma^2 u_i}+\log{(\tau^2+\sigma^2 u_i)}+c]$

with jacobian matrix (which cannot be calculated automatically in numpy):

$$\frac{\partial l(v)}{\partial \mu} = \sum \limits_{i=1}^{k} \frac{D_i - \mu}{\tau^2 + \sigma^2 u_i}$$

$$\frac{\partial l(v)}{\partial \sigma^2} = \sum \limits_{i=1}^{k} \frac{(D_i - \mu)^2 u_i}{(\tau^2 + \sigma^2 u_i)^2} - \sum \limits_{i=1}^{k} \frac{u_i}{\tau^2+\sigma^2u_i} \Rightarrow \frac{\partial l(v)}{\partial \sigma} = \frac{\partial l(v)}{\partial \sigma^2} \cdot 2 \sigma = (\sum \limits_{i=1}^{k} \frac{(D_i - \mu)^2 u_i}{(\tau^2 + \sigma^2 u_i)^2} - \sum \limits_{i=1}^{k} \frac{u_i}{\tau^2+\sigma^2u_i}) \cdot 2 \sigma$$

$$\frac{\partial l(v)}{\partial \tau^2} = \sum \limits_{i=1}^{k} \frac{(D_i - \mu)^2}{(\tau^2 + \sigma^2 u_i)^2} - \sum \limits_{i=1}^{k} \frac{1}{\tau^2+\sigma^2 u_i} \Rightarrow \frac{\partial l(v)}{\partial \tau} = \frac{\partial l(v)}{\partial \tau^2} \cdot 2 \tau = (\sum \limits_{i=1}^{k} \frac{(D_i - \mu)^2}{(\tau^2 + \sigma^2 u_i)^2} - \sum \limits_{i=1}^{k} \frac{1}{\tau^2+\sigma^2u_i}) \cdot 2 \tau$$

In [29]:
# define the negative log likelihood function
# minimize negative log likelihood function -> maximize log likelihood function
def nll(x, D, u):
    # x = np.array([mu, sigma, tau])
    mu, sigma, tau = x[0], x[1], x[2]
    sigma_square, tau_square = sigma**2, tau**2
    return 1/2*np.sum((D - mu)**2/(tau_square + sigma_square*u) \
            +np.log(tau_square + sigma_square*u)+np.log(2*pi))

# Jacobian: the matrix of all its first-order partial derivatives
def jac(x, D, u):
    mu, sigma, tau = x[0], x[1], x[2]
    sigma_square, tau_square = sigma**2, tau**2
    a = D - mu
    b = tau_square + sigma_square * u
    # convert the derivative with respect to sigma/tau, instead of sigma^2/tau^2
    d_mu = np.sum(a / b)
    d_sigma = (np.sum((a**2 * u) / b**2) - np.sum(u / b)) * (2*sigma) 
    d_tau = (np.sum(a**2 / b**2) - np.sum(1./ b)) * (2*tau)
    return - np.array([d_mu, d_sigma, d_tau])

def func(x, D, u):
    return nll(x, D, u), jac(x, D, u)

# mse(theta_hat, theta) = Var(theta_hat) + bias(theta_hat, theta)^2
def mse(param_ast,param):
    bias = np.mean(param_ast)-param
    mse = np.var(param_ast) + bias**2
    return mse

# summarize all the needed statistical infomation: [mean, bias, standard_deviation]
def stats_info(param_ast,param):
    result_list = [np.mean(param_ast), np.mean(param_ast)-param, np.std(param_ast), mse(param_ast,param)]
    return result_list

#############

def simulation(k, mu,sigma,tau,num_replications=10000):
    # store the results of convergent mu/sigma/tau for each replication
    all_mu_ast, all_sigma_square_ast, all_tau_square_ast = [], [], []

    for i in range(num_replications):
        # generate k(the number of studies) D_i
        u_array = uniform(0.02,0.2,k)
        u_inv_array = 1 / u_array
        # x_i ~ N(mu,tau^2)
        x_array = normal(mu,tau,k)
        # D_i ~ N(x_i, sigma^2*u_i)
        D_array = normal(x_array,sigma*u_array**0.5,k)

        # Note that we cannot use the starting values suggested in Sangnawakij's paper
        # since given tau^2, sigma^2 and mu are calculated under the condition that the derivatives are zero
        # which leads to a stationary point

        # random but resonable initial point
        mu_0 = uniform(-2.0, 2.0)
        sigma_0 = uniform(0., 15.)
        tau_0 = uniform(0., 8.)
        x0 = np.array([mu_0, sigma_0, tau_0])
        minimizer = minimize(nll, x0, args=(D_array, u_array), method="BFGS", jac=jac, tol=1e-10)   
        #minimizer_kwargs = {"method": "BFGS", "args": (D_array, u_array), "jac":True}
        # corresponding mu/sigma/tau when the negative log likelihood function is minimized
        mu_ast, sigma_ast, tau_ast = minimizer.x
        sigma_square_ast = sigma_ast**2
        tau_square_ast = tau_ast**2
        # print("mu: {0}, sigma_square: {1}, tau_square: {2}".format(mu_ast, sigma_square_ast, tau_square_ast))
        all_mu_ast.append(mu_ast)
        all_sigma_square_ast.append(sigma_square_ast)
        all_tau_square_ast.append(tau_square_ast)

    # convert the list to numpy array
    all_mu_ast = np.array(all_mu_ast)
    all_sigma_square_ast = np.array(all_sigma_square_ast)
    all_tau_square_ast = np.array(all_tau_square_ast)
    
    # return the result for mu/sigma/tau in the form of: [mean, bias, std]
    mu_info = stats_info(all_mu_ast,mu)
    sigma_square_info = stats_info(all_sigma_square_ast,sigma_square)
    tau_square_info = stats_info(all_tau_square_ast,tau_square)
    
    return mu_info, sigma_square_info, tau_square_info


In [30]:
%%time
#(mu, sigma^2,tau^2): (0,12,4),(0,9,4),(0,4,4),(0,2,6)
mu_mean, mu_bias, mu_se, mu_mse = [],[],[],[]
sigma_square_mean, sigma_square_bias, sigma_square_se, sigma_square_mse = [],[],[],[]
tau_square_mean, tau_square_bias, tau_square_se, tau_square_mse = [],[],[],[]

for para_combination in parameter_constellation:
    for k in k_list:
        # get the parameters
        mu = para_combination[0]
        sigma_square = para_combination[1]
        sigma = np.sqrt(sigma_square)
        tau_square = para_combination[2]
        tau = np.sqrt(tau_square)
        # simulate
        each_simulation = simulation(k, mu,sigma,tau,num_replications=10000)
        # append the results to lists
        mu_mean.append(each_simulation[0][0])
        mu_bias.append(each_simulation[0][1])
        mu_se.append(each_simulation[0][2])
        mu_mse.append(each_simulation[0][3])
        
        sigma_square_mean.append(each_simulation[1][0])
        sigma_square_bias.append(each_simulation[1][1])
        sigma_square_se.append(each_simulation[1][2]) 
        sigma_square_mse.append(each_simulation[1][3]) 
        
        tau_square_mean.append(each_simulation[2][0])
        tau_square_bias.append(each_simulation[2][1])
        tau_square_se.append(each_simulation[2][2])
        tau_square_mse.append(each_simulation[2][3])

#print('mean of mu_hat:', mu_mean)
#print('bias of mu_hat:', mu_bias)
#print('standard error of mu_hat:', mu_se)
#print('----------------------------------------------------------------')
#print('mean of sigma_square_hat:', sigma_square_mean)
#print('bias of sigma_square_hat:', sigma_square_bias)
#print('standard error of sigma_square_hat:', sigma_square_se)
#print('----------------------------------------------------------------')
#print('mean of tau_square_hat:', tau_square_mean)
#print('bias of tau_square_hat:', tau_square_bias)
#print('standard error of tau_square_hat:', tau_square_se)


CPU times: user 7min 20s, sys: 370 ms, total: 7min 20s
Wall time: 7min 20s


In [10]:
# from pd.dataframe to latex table
df = pd.DataFrame({'J_square': [i for i in J_square_list for _ in range(4)],
                   'k': k_list*4,
                   #'mean_mu': mu_mean,
                   #'mean(sigma^2)': sigma_square_mean,
                   #'mean(tau^2)': tau_square_mean,
                   'bias(mu)': mu_bias,
                   'bias(sigma^2)': sigma_square_bias,
                   'bias(tau^2)': tau_square_bias,
                   'standard_error(mu)': mu_se,
                   'standard_error(sigma^2)': sigma_square_se,
                   'standard_error(tau^2)': tau_square_se,
                   'mean_square_error(mu)': mu_mse,
                   'mean_square_error(sigma^2)': sigma_square_mse,
                   'mean_square_error(tau^2)': tau_square_mse,
                  })
#print(df.to_latex(index=False))

### simulation results for method 1 (BFGS)

|      J^2 |   k |  bias(mu) | bias(sigma^2) | bias(tau^2) | standard_error(mu) | standard_error(sigma^2) | standard_error(tau^2) | mean_square_error(mu) | mean_square_error(sigma^2) | mean_square_error(tau^2) |
|---------:|----:|----------:|--------------:|------------:|-------------------:|------------------------:|----------------------:|----------------------:|---------------------------:|-------------------------:|
| 0.250000 |  10 |  0.015063 |      8.383591 |   -1.490120 |           0.753766 |               24.201710 |              2.610821 |              0.568390 |                 656.007347 |                 9.036846 |
| 0.250000 |  30 | -0.005013 |      5.792728 |   -0.792221 |           0.434011 |               19.509819 |              2.102713 |              0.188390 |                 414.188733 |                 5.049017 |
| 0.250000 |  50 |  0.001728 |      3.845379 |   -0.517918 |           0.334190 |               16.553290 |              1.803792 |              0.111686 |                 288.798358 |                 3.521904 |
| 0.250000 | 100 | -0.000220 |      1.784520 |   -0.229929 |           0.232979 |               12.784251 |              1.393115 |              0.054279 |                 166.621580 |                 1.993637 |
| 0.307692 |  10 | -0.003276 |      9.552600 |   -1.538993 |           0.728910 |               22.687267 |              2.477987 |              0.531321 |                 605.964239 |                 8.508917 |
| 0.307692 |  30 | -0.000412 |      6.648014 |   -0.860767 |           0.409618 |               18.019787 |              1.979923 |              0.167787 |                 368.908804 |                 4.661016 |
| 0.307692 |  50 |  0.008128 |      4.442597 |   -0.568267 |           0.321619 |               15.031557 |              1.657667 |              0.103505 |                 245.684367 |                 3.070787 |
| 0.307692 | 100 | -0.001380 |      2.293977 |   -0.270609 |           0.224819 |               11.542149 |              1.278657 |              0.050546 |                 138.483531 |                 1.708193 |
| 0.500000 |  10 |  0.008040 |     11.439535 |   -1.696900 |           0.689693 |               19.696014 |              2.241551 |              0.475740 |                 518.795913 |                 7.904020 |
| 0.500000 |  30 | -0.000074 |      7.438150 |   -0.953367 |           0.391560 |               14.879255 |              1.742591 |              0.153319 |                 276.718303 |                 3.945532 |
| 0.500000 |  50 | -0.004299 |      5.674597 |   -0.693834 |           0.341112 |               12.543144 |              2.003931 |              0.116376 |                 189.531505 |                 4.497145 |
| 0.500000 | 100 |  0.000612 |      3.518768 |   -0.422142 |           0.213832 |                9.174100 |              1.078905 |              0.045724 |                  96.545828 |                 1.342240 |
| 0.750000 |  10 | -0.008572 |     18.670096 |   -2.641610 |           0.811979 |               27.889962 |              3.271412 |              0.659383 |                1126.422485 |                17.680241 |
| 0.750000 |  30 |  0.000854 |     12.330323 |   -1.548498 |           0.467274 |               20.175082 |              2.419755 |              0.218346 |                 559.070818 |                 8.253059 |
| 0.750000 |  50 |  0.001497 |      9.815208 |   -1.174408 |           0.355601 |               16.661566 |              1.982567 |              0.126454 |                 373.946099 |                 5.309807 |
| 0.750000 | 100 | -0.000879 |      6.269390 |   -0.727324 |           0.255918 |               11.888351 |              1.442181 |              0.065495 |                 180.638152 |                 2.608887 |

### method 1: bounded optimisation using tau^2 and sigma^2
### (comparison to the code currently used in PyMARE)


In [103]:
def simulation_bounds(k, mu,sigma,tau,num_replications=10000):
    # store the results of convergent mu/sigma/tau for each replication
    all_mu_ast, all_sigma_square_ast, all_tau_square_ast = [], [], []

    for i in range(num_replications):
        # generate k(the number of studies) D_i
        u_array = uniform(0.02,0.2,k)
        u_inv_array = 1 / u_array
        # x_i ~ N(mu,tau^2)
        x_array = normal(mu,tau,k)
        # D_i ~ N(x_i, sigma^2*u_i)
        D_array = normal(x_array,sigma*u_array**0.5,k)

        # Note that we cannot use the starting values suggested in Sangnawakij's paper
        # since given tau^2, sigma^2 and mu are calculated under the condition that the derivatives are zero
        # which leads to a stationary point

        # random but resonable initial point
        mu_0 = uniform(-2.0, 2.0)
        sigma_0 = uniform(0., 15.)
        tau_0 = uniform(0., 8.)
        x0 = np.array([mu_0, sigma_0, tau_0])
        bds = ((-np.inf, np.inf), (0, np.inf), (0, np.inf))
        minimizer = minimize(nll, x0, args=(D_array, u_array), jac=jac, bounds=bds, tol=1e-10)   
        #minimizer_kwargs = {"method": "BFGS", "args": (D_array, u_array), "jac":True}
        # corresponding mu/sigma/tau when the negative log likelihood function is minimized
        mu_ast, sigma_ast, tau_ast = minimizer.x
        sigma_square_ast = sigma_ast**2
        tau_square_ast = tau_ast**2
        
        #print('in the {}th replication'.format(i))
        #print('--------------------------------------------------------------')
        #print("mu: {0}, sigma_square: {1}, tau_square: {2}".format(mu_ast, sigma_square_ast, tau_square_ast))
        
        all_mu_ast.append(mu_ast)
        all_sigma_square_ast.append(sigma_square_ast)
        all_tau_square_ast.append(tau_square_ast)

    # convert the list to numpy array
    all_mu_ast = np.array(all_mu_ast)
    all_sigma_square_ast = np.array(all_sigma_square_ast)
    all_tau_square_ast = np.array(all_tau_square_ast)
    
    # return the result for mu/sigma/tau in the form of: [mean, bias, std]
    mu_info = stats_info(all_mu_ast,mu)
    sigma_square_info = stats_info(all_sigma_square_ast,sigma_square)
    tau_square_info = stats_info(all_tau_square_ast,tau_square)
    
    return mu_info, sigma_square_info, tau_square_info, all_mu_ast, all_sigma_square_ast, all_tau_square_ast


In [109]:
%%time
#(mu, sigma^2,tau^2): (0,12,4),(0,9,4),(0,4,4),(0,2,6)
mu_mean_bd, mu_bias_bd, mu_se_bd, mu_mse_bd = [],[],[],[]
sigma_square_mean_bd, sigma_square_bias_bd, sigma_square_se_bd, sigma_square_mse_bd = [],[],[],[]
tau_square_mean_bd, tau_square_bias_bd, tau_square_se_bd, tau_square_mse_bd = [],[],[],[]
mu_nonzero_bd,sigma_square_nonzero_bd, tau_square_nonzero_bd = [],[],[]

for para_combination in parameter_constellation:
    for k in k_list:
        # get the parameters
        mu = para_combination[0]
        sigma_square = para_combination[1]
        sigma = np.sqrt(sigma_square)
        tau_square = para_combination[2]
        tau = np.sqrt(tau_square)
        # simulate
        each_simulation = simulation_bounds(k, mu,sigma,tau,num_replications=10000)
        # append the results to lists
        mu_mean_bd.append(each_simulation[0][0])
        mu_bias_bd.append(each_simulation[0][1])
        mu_se_bd.append(each_simulation[0][2])
        mu_mse_bd.append(each_simulation[0][3])
        mu_nonzero_bd.append(np.count_nonzero(each_simulation[3])/10000)
        
        
        sigma_square_mean_bd.append(each_simulation[1][0])
        sigma_square_bias_bd.append(each_simulation[1][1])
        sigma_square_se_bd.append(each_simulation[1][2]) 
        sigma_square_mse_bd.append(each_simulation[1][3]) 
        sigma_square_nonzero_bd.append(np.count_nonzero(each_simulation[4])/10000)
        
        tau_square_mean_bd.append(each_simulation[2][0])
        tau_square_bias_bd.append(each_simulation[2][1])
        tau_square_se_bd.append(each_simulation[2][2])
        tau_square_mse_bd.append(each_simulation[2][3])
        tau_square_nonzero_bd.append(np.count_nonzero(each_simulation[5])/10000)

/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in true_divide
  
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in add
  
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in true_divide
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide b

CPU times: user 3min 48s, sys: 544 ms, total: 3min 49s
Wall time: 3min 49s


In [110]:
print('The average percentage of nonzero mu:', np.mean(mu_nonzero_bd))
print('The average percentage of nonzero sigma^2:', np.mean(sigma_square_nonzero_bd))
print('The average percentage of nonzero tau^2:', np.mean(tau_square_nonzero_bd))

The average percentage of nonzero mu: 1.0
The average percentage of nonzero sigma^2: 0.7381562500000001
The average percentage of nonzero tau^2: 0.6814312499999999


In [112]:
# from pd.dataframe to latex table
df_bd = pd.DataFrame({'J_square': [i for i in J_square_list for _ in range(4)],
                   'k': k_list*4,
                   #'mean_mu': mu_mean,
                   #'mean(sigma^2)': sigma_square_mean,
                   #'mean(tau^2)': tau_square_mean,
                   'bias(mu)': mu_bias_bd,
                   'bias(sigma^2)': sigma_square_bias_bd,
                   'bias(tau^2)': tau_square_bias_bd,
                   'standard_error(mu)': mu_se_bd,
                   'standard_error(sigma^2)': sigma_square_se_bd,
                   'standard_error(tau^2)': tau_square_se_bd,
                   'mean_square_error(mu)': mu_mse_bd,
                   'mean_square_error(sigma^2)': sigma_square_mse_bd,
                   'mean_square_error(tau^2)': tau_square_mse_bd,
                  })
#print(df_bd.to_latex(index=False))

|      J^2 |   k |  bias(mu) | bias(sigma^2) | bias(tau^2) | standard_error(mu) | standard_error(sigma^2) | standard_error(tau^2) | mean_square_error(mu) | mean_square_error(sigma^2) | mean_square_error(tau^2) |
|---------:|----:|----------:|--------------:|------------:|-------------------:|------------------------:|----------------------:|----------------------:|---------------------------:|-------------------------:|
| 0.250000 |  10 | -0.003582 |     17.652757 |   -2.231710 |           0.763582 |               26.940966 |              2.596801 |              0.583071 |                1037.435476 |                11.723908 |
| 0.250000 |  30 |  0.002199 |     16.324479 |   -1.623265 |           0.445366 |               25.325613 |              2.370924 |              0.198356 |                 907.875285 |                 8.256268 |
| 0.250000 |  50 | -0.004222 |     12.772508 |   -1.223818 |           0.343128 |               23.483373 |              2.159557 |              0.117754 |                 714.605744 |                 6.161416 |
| 0.250000 | 100 | -0.000187 |      8.168547 |   -0.679131 |           0.238726 |               20.098739 |              1.893699 |              0.056990 |                 470.684478 |                 4.047315 |
| 0.307692 |  10 | -0.004016 |     19.265076 |   -2.309305 |           0.746108 |               26.108918 |              2.495350 |              0.556693 |                1052.818760 |                11.559661 |
| 0.307692 |  30 | -0.001541 |     17.069612 |   -1.677353 |           0.427428 |               24.481948 |              2.271574 |              0.182697 |                 890.737443 |                 7.973559 |
| 0.307692 |  50 |  0.002937 |     13.524519 |   -1.289897 |           0.326139 |               22.851239 |              2.079932 |              0.106375 |                 705.091747 |                 5.989953 |
| 0.307692 | 100 |  0.003878 |      9.051139 |   -0.696109 |           0.238531 |               20.093939 |              1.883037 |              0.056912 |                 485.689488 |                 4.030395 |
| 0.500000 |  10 |  0.002501 |     21.349045 |   -2.440305 |           0.717296 |               24.140664 |              2.270557 |              0.514519 |                1038.553374 |                11.110519 |
| 0.500000 |  30 | -0.001735 |     19.395166 |   -1.838315 |           0.414937 |               23.473306 |              2.098042 |              0.172176 |                 927.168560 |                 7.781182 |
| 0.500000 |  50 |  0.001663 |     15.555830 |   -1.448763 |           0.317301 |               21.809961 |              1.943744 |              0.100683 |                 717.658250 |                 5.877053 |
| 0.500000 | 100 |  0.005736 |     11.009064 |   -0.812993 |           0.233060 |               19.807527 |              1.846691 |              0.054350 |                 513.537612 |                 4.071225 |
| 0.750000 |  10 | -0.001977 |     28.024747 |   -3.404922 |           0.844203 |               31.234867 |              3.284733 |              0.712683 |                1761.003335 |                22.382965 |
| 0.750000 |  30 |  0.003662 |     23.350686 |   -2.398181 |           0.477670 |               29.068647 |              2.887981 |              0.228183 |                1390.240785 |                14.091709 |
| 0.750000 |  50 |  0.004222 |     18.310000 |   -1.830981 |           0.359496 |               26.494248 |              2.577290 |              0.129255 |                1037.201311 |                 9.994915 |
| 0.750000 | 100 |  0.003698 |     12.023143 |   -1.163890 |           0.253093 |               22.012912 |              2.050227 |              0.064070 |                 629.124238 |                 5.558072 |

# Conclusion
1. the method of using BFGS on the unconstrained problem performs better, in terms of bias and standard deviation of the paremeters mu/sigma^2/tau^2
2. but the method of using bound 0 (to make sure sigma^2/tau^2 is positive) is computationally cheaper
3. only 73.82% of sigma^2 and 68.14% of tau^2 are convergent values (the remaining are enforced to be 0)

### simulation results in Sangnawakij's paper

In [19]:
mu_bias_paper = [0.0145, -0.0046, -0.0023, 0.0018, 0.0082, -0.0012, -0.0045, -0.0008, 0.0003, -0.0026, -0.0053, 0.0020, 0.0018, 0.0055, -0.0025, 0.0022]
sigma_square_bias_paper = [-0.4827,-0.2121, 0.1452,-0.0648, -1.9485, -0.2538, 0.2592, -0.0180, -1.4300, -0.4813, -0.0896, -0.0845, -1.3170, 0.1001, -0.1284, -0.0435]
tau_square_bias_paper = [-0.4039, -0.1411, -0.1023, -0.0508, -0.2987, -0.1226, -0.1056, -0.0456, -0.2121, -0.0820, -0.0744, -0.0374, -0.2996, -0.1945, -0.0755, -0.0737]

mu_se_paper = [0.8985, 0.4454, 0.3316, 0.2305, 0.9003, 0.4355, 0.3245, 0.2251, 0.8379, 0.4166, 0.3066, 0.2104, 0.9702, 0.4940, 0.3624, 0.2508]
sigma_square_se_paper = [50.2959, 30.9276, 21.6385, 14.0151, 38.2609, 28.6170, 20.7052, 13.8505, 39.2310, 26.0939, 18.4738, 12.3303, 77.3782, 39.2915, 25.8234, 16.4556]
tau_square_se_paper = [5.5567,3.5410, 2.1109, 1.5253, 4.9630, 3.1680, 2.4068, 1.5707, 5.2741, 3.1109, 2.1327, 1.3812, 9.3725, 4.3828, 3.3537, 1.9859]


mu_mse_paper = [mu_bias_paper[i]**2+mu_se_paper[i]**2 for i in range(16)]
sigma_square_mse_paper = [sigma_square_bias_paper[i]**2+sigma_square_se_paper[i]**2 for i in range(16)]
tau_square_mse_paper = [tau_square_bias_paper[i]**2+tau_square_se_paper[i]**2 for i in range(16)]


In [21]:
df_paper = pd.DataFrame({'J_square': [i for i in J_square_list for _ in range(4)],
                   'k': k_list*4,
                   #'mean_mu': mu_mean,
                   #'mean(sigma^2)': sigma_square_mean,
                   #'mean(tau^2)': tau_square_mean,
                   'bias(mu)': mu_bias_paper,
                   'bias(sigma^2)': sigma_square_bias_paper,
                   'bias(tau^2)': tau_square_bias_paper,
                   'standard_error(mu)': mu_se_paper,
                   'standard_error(sigma^2)': sigma_square_se_paper,
                   'standard_error(tau^2)': tau_square_se_paper,
                   'mean_square_error(mu)': mu_mse_paper,
                   'mean_square_error(sigma^2)': sigma_square_mse_paper,
                   'mean_square_error(tau^2)': tau_square_mse_paper})
#print(df_paper.to_latex(index=False))

|      J^2 |   k | bias(mu) | bias(sigma^2) | bias(tau^2) | standard_error(mu) | standard_error(sigma^2) | standard_error(tau^2) | mean_square_error(mu) | mean_square_error(sigma^2) | mean_square_error(tau^2) |
|---------:|----:|---------:|--------------:|------------:|-------------------:|------------------------:|----------------------:|----------------------:|---------------------------:|-------------------------:|
| 0.250000 |  10 |   0.0145 |       -0.4827 |     -0.4039 |             0.8985 |                 50.2959 |                5.5567 |              0.807512 |                2529.910556 |                31.040050 |
| 0.250000 |  30 |  -0.0046 |       -0.2121 |     -0.1411 |             0.4454 |                 30.9276 |                3.5410 |              0.198402 |                 956.561428 |                12.558590 |
| 0.250000 |  50 |  -0.0023 |        0.1452 |     -0.1023 |             0.3316 |                 21.6385 |                2.1109 |              0.109964 |                 468.245765 |                 4.466364 |
| 0.250000 | 100 |   0.0018 |       -0.0648 |     -0.0508 |             0.2305 |                 14.0151 |                1.5253 |              0.053133 |                 196.427227 |                 2.329121 |
| 0.307692 |  10 |   0.0082 |       -1.9485 |     -0.2987 |             0.9003 |                 38.2609 |                4.9630 |              0.810607 |                1467.693121 |                24.720591 |
| 0.307692 |  30 |  -0.0012 |       -0.2538 |     -0.1226 |             0.4355 |                 28.6170 |                3.1680 |              0.189662 |                 818.997103 |                10.051255 |
| 0.307692 |  50 |  -0.0045 |        0.2592 |     -0.1056 |             0.3245 |                 20.7052 |                2.4068 |              0.105321 |                 428.772492 |                 5.803838 |
| 0.307692 | 100 |  -0.0008 |       -0.0180 |     -0.0456 |             0.2251 |                 13.8505 |                1.5707 |              0.050671 |                 191.836674 |                 2.469178 |
| 0.500000 |  10 |   0.0003 |       -1.4300 |     -0.2121 |             0.8379 |                 39.2310 |                5.2741 |              0.702076 |                1541.116261 |                27.861117 |
| 0.500000 |  30 |  -0.0026 |       -0.4813 |     -0.0820 |             0.4166 |                 26.0939 |                3.1109 |              0.173562 |                 681.123267 |                 9.684423 |
| 0.500000 |  50 |  -0.0053 |       -0.0896 |     -0.0744 |             0.3066 |                 18.4738 |                2.1327 |              0.094032 |                 341.289315 |                 4.553945 |
| 0.500000 | 100 |   0.0020 |       -0.0845 |     -0.0374 |             0.2104 |                 12.3303 |                1.3812 |              0.044272 |                 152.043438 |                 1.909112 |
| 0.750000 |  10 |   0.0018 |       -1.3170 |     -0.2996 |             0.9702 |                 77.3782 |                9.3725 |              0.941291 |                5989.120324 |                87.933516 |
| 0.750000 |  30 |   0.0055 |        0.1001 |     -0.1945 |             0.4940 |                 39.2915 |                4.3828 |              0.244066 |                1543.831992 |                19.246766 |
| 0.750000 |  50 |  -0.0025 |       -0.1284 |     -0.0755 |             0.3624 |                 25.8234 |                3.3537 |              0.131340 |                 666.864474 |                11.253004 |
| 0.750000 | 100 |   0.0022 |       -0.0435 |     -0.0737 |             0.2508 |                 16.4556 |                1.9859 |              0.062905 |                 270.788664 |                 3.949231 |

# Conclusion
1. the simulation results in Sangnawakij's paper are better for bias, but have much higher standard deviation and mean square error (perhaps just simply ignore the negative sigma^2/tau^2?)

## method2: trying to calculate the maximum log likehood estimates of mu/sigma/tau iteratively (fixed point algorithm in Sangnawakij's paper)

### starting value
$$\tau_0^2 = 0$$
$$\hat{\mu_0} = \frac{\sum \limits_{i=1}^{k} \frac{D_i}{u_i}}{\sum \limits_{i=1}^{k} \frac{1}{u_i}}$$
$$\hat{\sigma_0}^2 = \frac{1}{k} \sum \limits_{i=1}^{k} \frac{(D_i - \hat{\mu_0})^2}{u_i}$$

### iterative steps
$$\mu_{s+1} = \frac{\sum \limits_{i=1}^{k}\frac{D_i}{\tau_s^2+\sigma_s^2 u_i}}{\sum \limits_{i=1}^{k} \frac{1}{\tau_s^2+\sigma_s^2 u_i}}$$
$$\sigma_{s+1}^2 = \frac{\sum \limits_{i=1}^{k} \frac{(D_i - \mu_s)^2 u_i - \tau_s^2 u_i}{(\tau_s^2 + \sigma_s^2 u_i)^2}}{\sum \limits_{i=1}^{k}\frac{u_i^2}{(\tau_s^2+\sigma_s^2 u_i)^2}} $$
$$\tau_{s+1}^2 = \frac{\sum \limits_{i=1}^{k} \frac{(D_i - \mu_s)^2 - \sigma_s^2 u_i}{(\tau_s^2 + \sigma_s^2 u_i)^2}}{\sum \limits_{i=1}^{k}\frac{1}{(\tau_s^2+\sigma_s^2 u_i)^2}} $$

In [54]:
def simulation_fp(k, actual_mu,actual_sigma,actual_tau,num_replications=10000,stopping_criteria = 1e-10,max_iter=1000):
    actual_sigma_square = actual_sigma**2
    actual_tau_square = actual_tau**2
    
    count_array = np.array([])
    mu_array = np.array([])
    tau_square_array = np.array([])
    sigma_square_array = np.array([])

    for num in range(num_replications):
        # firstly, generate all u_i and D_i from normal distribution
        u_array = uniform(0.02,0.2,k)
        # xi_i ~ N(mu,tau^2)
        xi_array = normal(actual_mu,actual_tau,k)
        # D_i ~ N(x_i, sigma^2*u_i)
        D_array = normal(xi_array,actual_sigma*u_array**0.5,k)
        u_inv_array = 1/u_array

        # starting value
        tau_square_hat_0 = 0
        mu_hat_0 = np.sum(D_array*u_inv_array)/np.sum(u_inv_array)
        sigma_square_hat_0 = 1/k*np.sum((D_array - mu_hat_0)**2/u_array) 

        log_likelihood = -1/2*np.sum((D_array - mu_hat_0)**2/(tau_square_hat_0 + sigma_square_hat_0*u_array)+np.log(tau_square_hat_0 + sigma_square_hat_0*u_array)+np.log(2*pi))
        #print('The log likelihood corresponding to starting values is', log_likelihood)

        mu = mu_hat_0
        tau = np.sqrt(tau_square_hat_0)
        sigma= np.sqrt(sigma_square_hat_0)



        #########
        count = 0
        error = 1
        while (error > stopping_criteria) and (count < max_iter):
            tau_square = tau**2
            sigma_square = sigma**2

            # calculate the common terms firstly to avoid repeated calculation
            common_term = 1/(tau_square + sigma_square*u_array)
            common_term_square = common_term**2
            D_minus_u_square = (D_array - mu)**2

            # iterative equations
            mu_updated = np.sum(D_array*common_term) / np.sum(common_term)
            sigma_square_updated = np.sum((D_minus_u_square*u_array - tau_square*u_array)*common_term_square)/np.sum(u_array**2*common_term_square)
            tau_square_updated = np.sum((D_minus_u_square - sigma_square*u_array)*common_term_square)/np.sum(common_term_square)
            log_likelihood = -1/2*np.sum((D_array - mu_updated)**2/(tau_square_updated + sigma_square_updated*u_array)+np.log(tau_square_updated + sigma_square_updated*u_array)+np.log(2*pi))

            
            # instead of discarding instances where sigma^2 or tau^2 is negative
            # continue to update other parameters while fixing one at zero
            if sigma_square_updated < 0:
                sigma_square_updated = 0 
            if tau_square_updated < 0:
                tau_square_updated = 0 
                
                
            # use sigma/tau -> unconstrained problem
            sigma_updated = np.sqrt(sigma_square_updated)
            tau_updated = np.sqrt(tau_square_updated)

            
            error = max(abs(mu_updated-mu),abs(sigma_updated-sigma),abs(tau_updated-tau))
            #print('max error = ',error)

            mu = mu_updated
            sigma = sigma_updated
            tau = tau_updated
            count += 1

        count_array = np.append(count_array, count)
        mu_array = np.append(mu_array, mu)
        mu_array_cleaned = [x for x in mu_array if str(x) != 'nan']
        
        sigma_square_array = np.append(sigma_square_array,sigma_square)
        tau_square_array = np.append(tau_square_array, tau_square)
            
    
    mu_info = stats_info(mu_array_cleaned,actual_mu)
    sigma_square_info = stats_info(sigma_square_array,actual_sigma_square)
    tau_square_info = stats_info(tau_square_array,actual_tau_square)
    
    #mu_info = [np.mean(mu_array_cleaned), np.mean(mu_array_cleaned)-actual_mu, np.std(mu_array_cleaned)]
    #sigma_square_info = [np.mean(sigma_square_array),np.mean(sigma_square_array)-actual_sigma_square, np.std(sigma_square_array)]
    #tau_square_info = [np.mean(tau_square_array),np.mean(tau_square_array)-actual_tau_square, np.std(tau_square_array)]
    return mu_info, sigma_square_info, tau_square_info,mu_array
    


In [55]:
%%time

#(mu, sigma^2,tau^2): (0,12,4),(0,9,4),(0,4,4),(0,2,6)
mu_mean_fp, mu_bias_fp, mu_se_fp, mu_mse_fp = [],[],[],[]
sigma_square_mean_fp, sigma_square_bias_fp, sigma_square_se_fp, sigma_square_mse_fp = [],[],[],[]
tau_square_mean_fp, tau_square_bias_fp, tau_square_se_fp, tau_square_mse_fp = [],[],[],[]

for para_combination in parameter_constellation:
    for k in k_list:
        # get the parameters
        actual_mu = para_combination[0]
        actual_sigma_square = para_combination[1]
        actual_sigma = np.sqrt(actual_sigma_square)
        actual_tau_square = para_combination[2]
        actual_tau = np.sqrt(actual_tau_square)
        # simulate
        each_simulation_fp = simulation_fp(k, actual_mu,actual_sigma,actual_tau,num_replications=10000,stopping_criteria = 1e-10)
        # append the results to lists
        mu_mean_fp.append(each_simulation_fp[0][0])
        mu_bias_fp.append(each_simulation_fp[0][1])
        mu_se_fp.append(each_simulation_fp[0][2])
        mu_mse_fp.append(each_simulation_fp[0][3])
        
        sigma_square_mean_fp.append(each_simulation_fp[1][0])
        sigma_square_bias_fp.append(each_simulation_fp[1][1])
        sigma_square_se_fp.append(each_simulation_fp[1][2]) 
        sigma_square_mse_fp.append(each_simulation_fp[1][3]) 
        
        tau_square_mean_fp.append(each_simulation_fp[2][0])
        tau_square_bias_fp.append(each_simulation_fp[2][1])
        tau_square_se_fp.append(each_simulation_fp[2][2])
        tau_square_mse_fp.append(each_simulation_fp[2][3])



/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in log
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in double_scalars


CPU times: user 32min 34s, sys: 5.21 s, total: 32min 40s
Wall time: 32min 45s


In [58]:
# from pd.dataframe to latex table
df_fp = pd.DataFrame({'J_square': [i for i in J_square_list for _ in range(4)],
                   'k': k_list*4,
                   #'mean_mu': mu_mean,
                   #'mean(sigma^2)': sigma_square_mean,
                   #'mean(tau^2)': tau_square_mean,
                   'bias(mu)': mu_bias_fp,
                   'bias(sigma^2)': sigma_square_bias_fp,
                   'bias(tau^2)': tau_square_bias_fp,
                   'standard_error(mu)': mu_se_fp,
                   'standard_error(sigma^2)': sigma_square_se_fp,
                   'standard_error(tau^2)': tau_square_se_fp,
                   'mean_square_error(mu)': mu_mse_fp,
                   'mean_square_error(sigma^2)': sigma_square_mse_fp,
                   'mean_square_error(tau^2)': tau_square_mse_fp 
                     })
#print(df_fp.to_latex(index=False))

|      J^2 |   k |  bias(mu) | bias(sigma^2) | bias(tau^2) | standard_error(mu) | standard_error(sigma^2) | standard_error(tau^2) | mean_square_error(mu) | mean_square_error(sigma^2) | mean_square_error(tau^2) |
|---------:|----:|----------:|--------------:|------------:|-------------------:|------------------------:|----------------------:|----------------------:|---------------------------:|-------------------------:|
| 0.250000 |  10 | -0.003575 |      9.962192 |   -1.663308 |           0.756690 |               25.304962 |              2.547157 |              0.572592 |                 739.586401 |                 9.254603 |
| 0.250000 |  30 |  0.002392 |      5.371822 |   -0.839931 |           0.426584 |               19.039728 |              2.085787 |              0.181979 |                 391.367700 |                 5.055990 |
| 0.250000 |  50 | -0.002300 |      3.954938 |   -0.566688 |           0.329113 |               16.313701 |              1.804624 |              0.108320 |                 281.778376 |                 3.577804 |
| 0.250000 | 100 |  0.003206 |      1.881366 |   -0.269810 |           0.229287 |               12.389136 |              1.368949 |              0.052583 |                 157.030232 |                 1.946819 |
| 0.307692 |  10 |  0.000604 |     10.667106 |   -1.720719 |           0.722216 |               23.281425 |              2.404876 |              0.521596 |                 655.811872 |                 8.744301 |
| 0.307692 |  30 |  0.003583 |      6.560022 |   -0.951178 |           0.417177 |               17.892526 |              1.995720 |              0.174050 |                 363.176364 |                 4.887638 |
| 0.307692 |  50 | -0.000409 |      4.475825 |   -0.647119 |           0.318318 |               14.781076 |              1.676616 |              0.101327 |                 238.513229 |                 3.229806 |
| 0.307692 | 100 |  0.002503 |      2.086503 |   -0.294261 |           0.224284 |               11.001995 |              1.252941 |              0.050310 |                 125.397397 |                 1.656451 |
| 0.500000 |  10 | -0.002387 |     12.489579 |   -1.824518 |           0.686515 |               20.247347 |              2.225263 |              0.471309 |                 565.944626 |                 8.280662 |
| 0.500000 |  30 |  0.003219 |      7.681078 |   -1.057390 |           0.393601 |               15.020755 |              1.758973 |              0.154932 |                 284.622030 |                 4.212058 |
| 0.500000 |  50 | -0.000307 |      5.753779 |   -0.779251 |           0.300613 |               12.285411 |              1.476736 |              0.090368 |                 184.037293 |                 2.787982 |
| 0.500000 | 100 | -0.000980 |      3.374411 |   -0.414421 |           0.213470 |                8.876709 |              1.078908 |              0.045571 |                  90.182613 |                 1.335787 |
| 0.750000 |  10 | -0.001528 |     19.461616 |   -2.819037 |           0.828388 |               27.740871 |              3.163208 |              0.686228 |                1148.310399 |                17.952856 |
| 0.750000 |  30 | -0.008325 |     12.801423 |   -1.696395 |           0.465705 |               20.271557 |              2.479370 |              0.216950 |                 574.812470 |                 9.025031 |
| 0.750000 |  50 | -0.000657 |      9.358000 |   -1.232508 |           0.355688 |               15.885533 |              2.063099 |              0.126514 |                 339.922325 |                 5.775453 |
| 0.750000 | 100 |  0.000459 |      6.108274 |   -0.780094 |           0.253424 |               11.187325 |              1.461703 |              0.064224 |                 162.467249 |                 2.745123 |

# Conclusion
1. Using fixed point algorithm leads to sightly worse results than unconstrained BFGS method
2. but fixed point algorithm is much more computationally expensive

### fixed point algorithm but use sigma^2/tau^2 in the stopping criteria

In [50]:
def simulation_fp2(k, actual_mu,actual_sigma,actual_tau,num_replications=10000,stopping_criteria = 1e-10,max_iter=1000):
    actual_sigma_square = actual_sigma**2
    actual_tau_square = actual_tau**2
    
    count_array = np.array([])
    mu_array = np.array([])
    tau_square_array = np.array([])
    sigma_square_array = np.array([])

    for num in range(num_replications):
        # firstly, generate all u_i and D_i from normal distribution
        u_array = uniform(0.02,0.2,k)
        # xi_i ~ N(mu,tau^2)
        xi_array = normal(actual_mu,actual_tau,k)
        # D_i ~ N(x_i, sigma^2*u_i)
        D_array = normal(xi_array,actual_sigma*u_array**0.5,k)
        u_inv_array = 1/u_array

        # starting value
        tau_square_hat_0 = 0
        mu_hat_0 = np.sum(D_array*u_inv_array)/np.sum(u_inv_array)
        sigma_square_hat_0 = 1/k*np.sum((D_array - mu_hat_0)**2/u_array) 

        log_likelihood = -1/2*np.sum((D_array - mu_hat_0)**2/(tau_square_hat_0 + sigma_square_hat_0*u_array)+np.log(tau_square_hat_0 + sigma_square_hat_0*u_array)+np.log(2*pi))
        #print('The log likelihood corresponding to starting values is', log_likelihood)

        mu = mu_hat_0
        tau_square = tau_square_hat_0
        sigma_square = sigma_square_hat_0



        #########
        count = 0
        error = 1
        while (error > stopping_criteria) and (count < max_iter):

            # calculate the common terms firstly to avoid repeated calculation
            common_term = 1/(tau_square + sigma_square*u_array)
            common_term_square = common_term**2
            D_minus_u_square = (D_array - mu)**2

            # iterative equations
            mu_updated = np.sum(D_array*common_term) / np.sum(common_term)
            sigma_square_updated = np.sum((D_minus_u_square*u_array - tau_square*u_array)*common_term_square)/np.sum(u_array**2*common_term_square)
            tau_square_updated = np.sum((D_minus_u_square - sigma_square*u_array)*common_term_square)/np.sum(common_term_square)
            log_likelihood = -1/2*np.sum((D_array - mu_updated)**2/(tau_square_updated + sigma_square_updated*u_array)+np.log(tau_square_updated + sigma_square_updated*u_array)+np.log(2*pi))

            
            # instead of discarding instances where sigma^2 or tau^2 is negative
            # continue to update other parameters while fixing one at zero
            if (sigma_square_updated) < 0 or (np.isnan(sigma_square_updated)==True):
                sigma_square_updated = 0 
            if tau_square_updated < 0 or (np.isnan(tau_square_updated)==True):
                tau_square_updated = 0 

            
            error = max(abs(mu_updated-mu),abs(sigma_square_updated-sigma_square),abs(tau_square_updated-tau_square))
            #print('max error = ',error)

            mu = mu_updated
            sigma_square = sigma_square_updated
            tau_square = tau_square_updated
            count += 1

        count_array = np.append(count_array, count)
        mu_array = np.append(mu_array, mu)
        mu_array_cleaned = [x for x in mu_array if str(x) != 'nan']
        
        sigma_square_array = np.append(sigma_square_array,sigma_square)
        tau_square_array = np.append(tau_square_array, tau_square)
            
    
    mu_info = stats_info(mu_array_cleaned,actual_mu)
    sigma_square_info = stats_info(sigma_square_array,actual_sigma_square)
    tau_square_info = stats_info(tau_square_array,actual_tau_square)
    
    #mu_info = [np.mean(mu_array_cleaned), np.mean(mu_array_cleaned)-actual_mu, np.std(mu_array_cleaned)]
    #sigma_square_info = [np.mean(sigma_square_array),np.mean(sigma_square_array)-actual_sigma_square, np.std(sigma_square_array)]
    #tau_square_info = [np.mean(tau_square_array),np.mean(tau_square_array)-actual_tau_square, np.std(tau_square_array)]
    return mu_info, sigma_square_info, tau_square_info,mu_array
    


In [51]:
%%time

#(mu, sigma^2,tau^2): (0,12,4),(0,9,4),(0,4,4),(0,2,6)
mu_mean_fp2, mu_bias_fp2, mu_se_fp2, mu_mse_fp2 = [],[],[],[]
sigma_square_mean_fp2, sigma_square_bias_fp2, sigma_square_se_fp2, sigma_square_mse_fp2 = [],[],[],[]
tau_square_mean_fp2, tau_square_bias_fp2, tau_square_se_fp2, tau_square_mse_fp2 = [],[],[],[]

for para_combination in parameter_constellation:
    for k in k_list:
        # get the parameters
        actual_mu = para_combination[0]
        actual_sigma_square = para_combination[1]
        actual_sigma = np.sqrt(actual_sigma_square)
        actual_tau_square = para_combination[2]
        actual_tau = np.sqrt(actual_tau_square)
        # simulate
        each_simulation_fp2 = simulation_fp2(k, actual_mu,actual_sigma,actual_tau,num_replications=10000,stopping_criteria = 1e-10)
        # append the results to lists
        mu_mean_fp2.append(each_simulation_fp2[0][0])
        mu_bias_fp2.append(each_simulation_fp2[0][1])
        mu_se_fp2.append(each_simulation_fp2[0][2])
        mu_mse_fp2.append(each_simulation_fp2[0][3])
        
        sigma_square_mean_fp2.append(each_simulation_fp2[1][0])
        sigma_square_bias_fp2.append(each_simulation_fp2[1][1])
        sigma_square_se_fp2.append(each_simulation_fp2[1][2]) 
        sigma_square_mse_fp2.append(each_simulation_fp2[1][3]) 
        
        tau_square_mean_fp2.append(each_simulation_fp2[2][0])
        tau_square_bias_fp2.append(each_simulation_fp2[2][1])
        tau_square_se_fp2.append(each_simulation_fp2[2][2])
        tau_square_mse_fp2.append(each_simulation_fp2[2][3])

/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in log
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


CPU times: user 34min 28s, sys: 5.44 s, total: 34min 33s
Wall time: 34min 39s


In [53]:
# from pd.dataframe to latex table
df_fp2 = pd.DataFrame({'J_square': [i for i in J_square_list for _ in range(4)],
                   'k': k_list*4,
                   #'mean_mu': mu_mean,
                   #'mean(sigma^2)': sigma_square_mean,
                   #'mean(tau^2)': tau_square_mean,
                   'bias(mu)': mu_bias_fp2,
                   'bias(sigma^2)': sigma_square_bias_fp2,
                   'bias(tau^2)': tau_square_bias_fp2,
                   'standard_error(mu)': mu_se_fp2,
                   'standard_error(sigma^2)': sigma_square_se_fp2,
                   'standard_error(tau^2)': tau_square_se_fp2,
                   'mean_square_error(mu)': mu_mse_fp2,
                   'mean_square_error(sigma^2)': sigma_square_mse_fp2,
                   'mean_square_error(tau^2)': tau_square_mse_fp2 
                     })
#print(df_fp2.to_latex(index=False))

|      J^2 |   k |  bias(mu) | bias(sigma^2) | bias(tau^2) | standard_error(mu) | standard_error(sigma^2) | standard_error(tau^2) | mean_square_error(mu) | mean_square_error(sigma^2) | mean_square_error(tau^2) |
|---------:|----:|----------:|--------------:|------------:|-------------------:|------------------------:|----------------------:|----------------------:|---------------------------:|-------------------------:|
| 0.250000 |  10 | -0.004873 |      9.468296 |   -1.640197 |           0.743940 |               25.182772 |              2.553828 |              0.553471 |                 723.820659 |                 9.212281 |
| 0.250000 |  30 |  0.002921 |      5.497268 |   -0.875660 |           0.426794 |               19.076579 |              2.080083 |              0.182161 |                 394.135828 |                 5.093527 |
| 0.250000 |  50 |  0.000086 |      3.861925 |   -0.568285 |           0.330645 |               16.212076 |              1.802149 |              0.109326 |                 277.745867 |                 3.570689 |
| 0.250000 | 100 | -0.002667 |      1.782698 |   -0.269328 |           0.228486 |               12.408618 |              1.383210 |              0.052213 |                 157.151814 |                 1.985808 |
| 0.307692 |  10 |  0.001054 |     10.680689 |   -1.686792 |           0.734097 |               23.422520 |              2.442005 |              0.538900 |                 662.691536 |                 8.808656 |
| 0.307692 |  30 |  0.003264 |      6.542460 |   -0.948649 |           0.414142 |               17.682961 |              1.986852 |              0.171524 |                 355.490876 |                 4.847515 |
| 0.307692 |  50 |  0.002912 |      4.353908 |   -0.602391 |           0.317025 |               14.889334 |              1.679542 |              0.100513 |                 240.648784 |                 3.183736 |
| 0.307692 | 100 |  0.000734 |      2.278154 |   -0.312271 |           0.226748 |               11.206731 |              1.280015 |              0.051415 |                 130.780806 |                 1.735950 |
| 0.500000 |  10 | -0.000115 |     12.264351 |   -1.809824 |           0.688790 |               20.296978 |              2.223295 |              0.474432 |                 562.381628 |                 8.218506 |
| 0.500000 |  30 |  0.008859 |      7.781138 |   -1.055339 |           0.397392 |               15.055698 |              1.761012 |              0.157999 |                 287.220158 |                 4.214903 |
| 0.500000 |  50 |  0.002006 |      5.534228 |   -0.755026 |           0.300699 |               12.121108 |              1.485492 |              0.090424 |                 177.548933 |                 2.776751 |
| 0.500000 | 100 |  0.004121 |      3.330747 |   -0.427479 |           0.212189 |                8.775453 |              1.078195 |              0.045041 |                  88.102454 |                 1.345243 |
| 0.750000 |  10 | -0.003519 |     19.302413 |   -2.790113 |           0.823660 |               27.658465 |              3.167895 |              0.678428 |                1137.573835 |                17.820290 |
| 0.750000 |  30 |  0.000470 |     12.231375 |   -1.681202 |           0.458662 |               19.884437 |              2.446946 |              0.210371 |                 544.997344 |                 8.813982 |
| 0.750000 |  50 | -0.002729 |      9.190675 |   -1.215746 |           0.353009 |               15.940774 |              2.015028 |              0.124623 |                 338.576788 |                 5.538377 |
| 0.750000 | 100 |  0.001535 |      5.840597 |   -0.728906 |           0.252626 |               10.747381 |              1.445401 |              0.063822 |                 149.618771 |                 2.620487 |

# Conclusion
1. Using sigma^2/tau^2 (instead of sigma/tau) in the stopping criteria sightly reduces bias/mean square error

### fixed point algorithm but use log-likelihood in the stopping criteria

In [68]:
def simulation_fp3(k, actual_mu,actual_sigma,actual_tau,num_replications=10000,stopping_criteria = 1e-10,max_iter=1000):
    actual_sigma_square = actual_sigma**2
    actual_tau_square = actual_tau**2
    
    count_array = np.array([])
    mu_array = np.array([])
    tau_square_array = np.array([])
    sigma_square_array = np.array([])

    for num in range(num_replications):
        # firstly, generate all u_i and D_i from normal distribution
        u_array = uniform(0.02,0.2,k)
        # xi_i ~ N(mu,tau^2)
        xi_array = normal(actual_mu,actual_tau,k)
        # D_i ~ N(x_i, sigma^2*u_i)
        D_array = normal(xi_array,actual_sigma*u_array**0.5,k)
        u_inv_array = 1/u_array

        # starting value
        tau_square_hat_0 = 0
        mu_hat_0 = np.sum(D_array*u_inv_array)/np.sum(u_inv_array)
        sigma_square_hat_0 = 1/k*np.sum((D_array - mu_hat_0)**2/u_array) 

        log_likelihood = -1/2*np.sum((D_array - mu_hat_0)**2/(tau_square_hat_0 + sigma_square_hat_0*u_array)+np.log(tau_square_hat_0 + sigma_square_hat_0*u_array)+np.log(2*pi))
        #print('The log likelihood corresponding to starting values is', log_likelihood)

        mu = mu_hat_0
        tau_square = tau_square_hat_0
        sigma_square = sigma_square_hat_0



        #########
        count = 0
        error = 1
        while (error > stopping_criteria) and (count < max_iter):

            # calculate the common terms firstly to avoid repeated calculation
            common_term = 1/(tau_square + sigma_square*u_array)
            common_term_square = common_term**2
            D_minus_u_square = (D_array - mu)**2

            # iterative equations
            mu_updated = np.sum(D_array*common_term) / np.sum(common_term)
            sigma_square_updated = np.sum((D_minus_u_square*u_array - tau_square*u_array)*common_term_square)/np.sum(u_array**2*common_term_square)
            tau_square_updated = np.sum((D_minus_u_square - sigma_square*u_array)*common_term_square)/np.sum(common_term_square)
            log_likelihood_updated = -1/2*np.sum((D_array - mu_updated)**2/(tau_square_updated + sigma_square_updated*u_array)+np.log(tau_square_updated + sigma_square_updated*u_array)+np.log(2*pi))

            
            # instead of discarding instances where sigma^2 or tau^2 is negative
            # continue to update other parameters while fixing one at zero
            if (sigma_square_updated) < 0 or (np.isnan(sigma_square_updated)==True):
                sigma_square_updated = 0 
            if tau_square_updated < 0 or (np.isnan(tau_square_updated)==True):
                tau_square_updated = 0 

            
            error = abs(log_likelihood_updated-log_likelihood)
            #error = max(abs(mu_updated-mu),abs(sigma_square_updated-sigma_square),abs(tau_square_updated-tau_square))
            #print('max error = ',error)

            mu = mu_updated
            sigma_square = sigma_square_updated
            tau_square = tau_square_updated
            log_likelihood = log_likelihood_updated
            count += 1

        count_array = np.append(count_array, count)
        mu_array = np.append(mu_array, mu)
        mu_array_cleaned = [x for x in mu_array if str(x) != 'nan']
        
        sigma_square_array = np.append(sigma_square_array,sigma_square)
        tau_square_array = np.append(tau_square_array, tau_square)
            
    
    mu_info = stats_info(mu_array_cleaned,actual_mu)
    sigma_square_info = stats_info(sigma_square_array,actual_sigma_square)
    tau_square_info = stats_info(tau_square_array,actual_tau_square)
    
    #mu_info = [np.mean(mu_array_cleaned), np.mean(mu_array_cleaned)-actual_mu, np.std(mu_array_cleaned)]
    #sigma_square_info = [np.mean(sigma_square_array),np.mean(sigma_square_array)-actual_sigma_square, np.std(sigma_square_array)]
    #tau_square_info = [np.mean(tau_square_array),np.mean(tau_square_array)-actual_tau_square, np.std(tau_square_array)]
    return mu_info, sigma_square_info, tau_square_info
    


In [69]:
%%time

#(mu, sigma^2,tau^2): (0,12,4),(0,9,4),(0,4,4),(0,2,6)
mu_mean_fp3, mu_bias_fp3, mu_se_fp3, mu_mse_fp3 = [],[],[],[]
sigma_square_mean_fp3, sigma_square_bias_fp3, sigma_square_se_fp3, sigma_square_mse_fp3 = [],[],[],[]
tau_square_mean_fp3, tau_square_bias_fp3, tau_square_se_fp3, tau_square_mse_fp3 = [],[],[],[]

for para_combination in parameter_constellation:
    for k in k_list:
        # get the parameters
        actual_mu = para_combination[0]
        actual_sigma_square = para_combination[1]
        actual_sigma = np.sqrt(actual_sigma_square)
        actual_tau_square = para_combination[2]
        actual_tau = np.sqrt(actual_tau_square)
        # simulate
        each_simulation_fp3 = simulation_fp3(k, actual_mu,actual_sigma,actual_tau,num_replications=10000,stopping_criteria = 1e-10,max_iter=1000)
        # append the results to lists
        mu_mean_fp3.append(each_simulation_fp3[0][0])
        mu_bias_fp3.append(each_simulation_fp3[0][1])
        mu_se_fp3.append(each_simulation_fp3[0][2])
        mu_mse_fp3.append(each_simulation_fp3[0][3])
        
        sigma_square_mean_fp3.append(each_simulation_fp3[1][0])
        sigma_square_bias_fp3.append(each_simulation_fp3[1][1])
        sigma_square_se_fp3.append(each_simulation_fp3[1][2]) 
        sigma_square_mse_fp3.append(each_simulation_fp3[1][3]) 
        
        tau_square_mean_fp3.append(each_simulation_fp3[2][0])
        tau_square_bias_fp3.append(each_simulation_fp3[2][1])
        tau_square_se_fp3.append(each_simulation_fp3[2][2])
        tau_square_mse_fp3.append(each_simulation_fp3[2][3])

/opt/anaconda3/envs/tensorflow_cpu_1/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in log


CPU times: user 21min 57s, sys: 2.76 s, total: 21min 59s
Wall time: 22min 2s


In [73]:
# from pd.dataframe to latex table
df_fp_likelihood = pd.DataFrame({'J_square': [i for i in J_square_list for _ in range(4)],
                   'k': k_list*4,
                   #'mean_mu': mu_mean,
                   #'mean(sigma^2)': sigma_square_mean,
                   #'mean(tau^2)': tau_square_mean,
                   'bias(mu)': mu_bias_fp3,
                   'bias(sigma^2)': sigma_square_bias_fp3,
                   'bias(tau^2)': tau_square_bias_fp3,
                   'standard_error(mu)': mu_se_fp3,
                   'standard_error(sigma^2)': sigma_square_se_fp3,
                   'standard_error(tau^2)': tau_square_se_fp3,
                   'mean_square_error(mu)': mu_mse_fp3,
                   'mean_square_error(sigma^2)': sigma_square_mse_fp3,
                   'mean_square_error(tau^2)': tau_square_mse_fp3 
                     })
#print(df_fp_likelihood.to_latex(index=False))

|      J^2 |   k |  bias(mu) | bias(sigma^2) | bias(tau^2) | standard_error(mu) | standard_error(sigma^2) | standard_error(tau^2) | mean_square_error(mu) | mean_square_error(sigma^2) | mean_square_error(tau^2) |
|---------:|----:|----------:|--------------:|------------:|-------------------:|------------------------:|----------------------:|----------------------:|---------------------------:|-------------------------:|
| 0.250000 |  10 | -0.005174 |      9.711040 |   -1.712070 |           0.757128 |               24.791189 |              2.549894 |              0.573270 |                 708.907352 |                 9.433140 |
| 0.250000 |  30 |  0.001460 |      5.821724 |   -0.925972 |           0.429855 |               19.432036 |              2.115793 |              0.184778 |                 411.496486 |                 5.334004 |
| 0.250000 |  50 | -0.002359 |      4.010268 |   -0.622928 |           0.325691 |               16.221227 |              1.795142 |              0.106080 |                 279.210446 |                 3.610575 |
| 0.250000 | 100 | -0.000276 |      1.932409 |   -0.280675 |           0.231003 |               12.553971 |              1.395134 |              0.053363 |                 161.336392 |                 2.025177 |
| 0.307692 |  10 |  0.001529 |     10.893905 |   -1.788241 |           0.732325 |               23.170960 |              2.421535 |              0.536303 |                 655.570549 |                 9.061637 |
| 0.307692 |  30 |  0.001147 |      6.167275 |   -0.972832 |           0.414077 |               17.591485 |              1.963082 |              0.171461 |                 347.495611 |                 4.800092 |
| 0.307692 |  50 | -0.000528 |      4.171848 |   -0.655791 |           0.320221 |               14.563493 |              1.681874 |              0.102541 |                 229.499653 |                 3.258763 |
| 0.307692 | 100 | -0.000291 |      2.269190 |   -0.322941 |           0.229077 |               11.214623 |              1.274652 |              0.052476 |                 130.916985 |                 1.729030 |
| 0.500000 |  10 | -0.005658 |     12.173196 |   -1.852697 |           0.694146 |               20.326759 |              2.198216 |              0.481871 |                 561.363821 |                 8.264640 |
| 0.500000 |  30 |  0.003797 |      7.661674 |   -1.119661 |           0.400124 |               14.891545 |              1.785448 |              0.160114 |                 280.459368 |                 4.441467 |
| 0.500000 |  50 | -0.004670 |      5.567230 |   -0.797164 |           0.301772 |               12.293513 |              1.503841 |              0.091088 |                 182.124499 |                 2.897009 |
| 0.500000 | 100 | -0.002215 |      3.274089 |   -0.438493 |           0.211788 |                8.822833 |              1.102298 |              0.044859 |                  88.562035 |                 1.407337 |
| 0.750000 |  10 |  0.016463 |     19.926767 |   -2.905911 |           0.832966 |               28.584262 |              3.159829 |              0.694103 |                1214.136045 |                18.428837 |
| 0.750000 |  30 | -0.009901 |     12.263980 |   -1.787168 |           0.465899 |               20.026254 |              2.483525 |              0.217160 |                 551.456056 |                 9.361867 |
| 0.750000 |  50 | -0.005404 |      9.554296 |   -1.344698 |           0.356277 |               16.177613 |              2.090392 |              0.126962 |                 352.999738 |                 6.177953 |
| 0.750000 | 100 | -0.001712 |      6.119245 |   -0.800650 |           0.254635 |               11.173740 |              1.505209 |              0.064842 |                 162.297620 |                 2.906696 |

# Conclusion
1. Using the change of log-likehood function as the stopping criteria would lead to similar results, but computationally cheaper

In [ ]:
s